In [1]:
import torch
w = torch.load('/home/arda/dinov2/distillation/logs/resnet50/distillation/dino_base_448/checkpoints/last.ckpt')

/home/arda/tmp/ipykernel_2238048/2639256857.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  w = torch.load('/home/arda/dinov2/distillation/logs/resnet50/distillation/din

In [2]:
w_student = {k: v for k, v in w['state_dict'].items() if 'student' in k and 'scalekd' not in k and 'feature_matchers' not in k}

In [3]:
w_student.keys()

dict_keys(['student.model.model.stem.conv1.weight', 'student.model.model.stem.conv1.norm.weight', 'student.model.model.stem.conv1.norm.bias', 'student.model.model.stem.conv1.norm.running_mean', 'student.model.model.stem.conv1.norm.running_var', 'student.model.model.stem.conv1.norm.num_batches_tracked', 'student.model.model.res2.0.shortcut.weight', 'student.model.model.res2.0.shortcut.norm.weight', 'student.model.model.res2.0.shortcut.norm.bias', 'student.model.model.res2.0.shortcut.norm.running_mean', 'student.model.model.res2.0.shortcut.norm.running_var', 'student.model.model.res2.0.shortcut.norm.num_batches_tracked', 'student.model.model.res2.0.conv1.weight', 'student.model.model.res2.0.conv1.norm.weight', 'student.model.model.res2.0.conv1.norm.bias', 'student.model.model.res2.0.conv1.norm.running_mean', 'student.model.model.res2.0.conv1.norm.running_var', 'student.model.model.res2.0.conv1.norm.num_batches_tracked', 'student.model.model.res2.0.conv2.weight', 'student.model.model.res2

In [4]:
# Convert ResNet50 weights from student model to Detectron2 format\
import pickle as pkl
detectron2_weights = {}

# Map the keys from student model to Detectron2 format
for k, v in w_student.items():
    # Remove the 'student.model.model.' prefix
    new_k = k.replace('student.model.model.', 'backbone.')
    
    # Handle stem weights
    if 'stem' in new_k:
        new_k = new_k.replace('conv1', 'conv1')
    
    # Handle residual blocks
    for i in range(2, 6):
        new_k = new_k.replace(f'res{i}', f'res{i}')
    
    detectron2_weights[new_k] = v.detach().cpu().numpy()

# Save in Detectron2 compatible format with metadata
res = {
    "model": detectron2_weights,
    "__author__": "dinov2_distillation",
    "matching_heuristics": True
}

with open("/home/arda/anyma/pretrained_models/R-50_dino_l448.pkl", "wb") as f:
    pkl.dump(res, f)


: 